In [36]:
import numpy as np
import pandas as pd
import scipy
from numpy.linalg import inv, det
from sklearn.tree import DecisionTreeClassifier
from functools import partial
from scipy.stats import norm

First let's prepare the data.

In [37]:
with open('spam.txt', 'r') as f:
    arr = []
    for line in f:
        row = line.split()
        row = list(map(lambda x: float(x), row))
        arr.append(row)

df = pd.DataFrame(arr)
df[57] = df[57].astype(int)

ds = df.to_numpy()

X = ds[:,:-1]
Y = ds[:,-1]

In [38]:
np.random.seed(42)

We change the class labels from 0 and 1 to -1 and 1.

In [39]:
Y[Y == 0.] = -1.

In [40]:
def proportion_with_weights(Y, weights, indices):
    """
        Calculate the proportion of classes in Y in each
        class where the indices lie in indices.
        The proportion will actually be weighted by the array
        weights so that indices with higher weights will
        affect the proportion more.
        Y: a numpy array of class labels
        weights: a 1-d numpy array of weights with total sum 1
            and Y.shape = weights.shape
        indices: the indices to compute the proportions over
        returns:
            p0, p1: the weighted proportions
    """
    p0 = np.dot(Y[indices] == -1, weights[indices]) / len(Y[indices])
    p1 = np.dot(Y[indices] == 1, weights[indices]) / len(Y[indices])
    
    return p0, p1

In [41]:
def split_to_indices(X, Y, indices, col, t):
    """
        Calculate new indices obtained by splitting on the column col
        according to col <= t or col > t. The new indices are obtained
        by restricting (X,Y) to the rows specified by indices.
        X, Y: arrays with X.shape[0] = Y.shape[0]
        indices: a subset of the indices from 0 to Y.shape[0]
        col: the column specifying a feature to split on
        t: the value defining the split according to col <= or col > t
        Returns:
            new_indices: a pair of subsets of indices which
            partition indices when taken together
    """
    new_indices = (np.intersect1d(indices, np.where(X[:, col] <= t)),
            np.intersect1d(indices, np.where(X[:, col] > t)))
    return new_indices

In [42]:
def calc_impurity_with_weights(X, Y, weights, indices, col, t):
    """
        Calculate the Gini impurity (with weights) after splitting (X,Y)
        on the column col according to col <= t or col > t.
        X, Y: arrays with X.shape[0] = Y.shape[0]
        indices: a subset of the indices from 0 to Y.shape[0]
        weights: an array with weights.shape = Y.shape
        col: the column specifying a feature to split on
        t: the value defining the split according to col <= or col > t
    """
    
    new_indices = split_to_indices(X, Y, indices, col, t)
    
    gammas = []
    
    for new_ind in new_indices:
        p0, p1 = proportion_with_weights(Y, weights, new_ind)
        gamma = 1 - p0**2 - p1**2
        gammas.append(gamma)
    
    impurity = np.array(gammas).sum()
    
    return impurity

In [43]:
def generate_split_with_weights(X, Y, indices, weights, n_0=500, frac=4, num_ts=20):
    """
        Calculate a split to minimize the gini impurity.
        X, Y: arrays with X.shape[0] = Y.shape[0]
        indices: a subset of the indices from 0 to Y.shape[0]
        weights: an array with weights.shape = Y.shape
        n_0: do not compute a split if there are fewer than this
            number of indices in indices
        frac: only create a split in which both subsets of the
            partition of indices contain at least n_0 / frac elements
        num_ts: the number of values of t to test as in col <= t
            and col < t
        Returns:
            best_split: a partition of indices according to the best
            split
    """
    
    if len(indices) <= n_0:
        return (None, None)
    else:
        best_split = (None, None)
        min_impurity = float('inf')

        k = X.shape[1]
        for col in range(k):
            low = X[indices, col].min()
            high = X[indices, col].max()
            ts = np.linspace(low, high, num_ts)

            for t in ts:
                impurity = calc_impurity_with_weights(X, Y, weights, indices, col, t)
                new_indices = split_to_indices(X, Y, indices, col, t)
                if len(new_indices[0]) >= n_0 / frac and len(new_indices[1]) >= n_0 / frac:
                    if impurity < min_impurity:
                        min_impurity = impurity
                        best_split = (col, t)

        return best_split

In [44]:
class Node:
    """
        Represents a node of a decision tree.
    """
    
    def __init__(self, X, Y, indices, col=None, t=None, lchild=None, rchild=None):
        """
            (X, Y): the dataset to train on
            indices: the indices corresponding to the split
                represented by this node
            col: a column to split on, if the node has children
            t: the value for the split according to col <= t or col > t
            lchild, rchild: nodes representing the left and right child
        """
        self.X = X
        self.Y = Y
        self.avg = Y[indices].mean()
        self.indices = indices
        self.col = col
        self.t = t
        self.lchild = lchild
        self.rchild = rchild
    
    def split(self, weights, n_0, frac, num_ts):
        """
            Split the node using generate_split_with_weights and assign
            its children, column, and t
        """
        (col, t) = generate_split_with_weights(self.X, self.Y, self.indices, weights, n_0, frac, num_ts)
        if col is not None:
            new_indices = split_to_indices(self.X, self.Y, self.indices, col, t)
            self.col = col
            self.t = t
            self.lchild = Node(self.X, self.Y, new_indices[0])
            self.rchild = Node(self.X, self.Y, new_indices[1])

In [45]:
class DecisionTree:
    """
        Represents a decision tree.
    """
    
    def __init__(self, X, Y, weights=np.ones(len(Y)) / len(Y), n_0=500, frac=4, num_ts=20):
        """
            Initialize the tree with a single node.
            (X, Y): the dataset to train on
            weights: an array with weights.shape=Y.shape representing
                weights for the different examples of the dataset
            n_0: an integer such that nodes of size <= n_0 are not split
                further
            frac: an integer such that each node has size >= n_0/frac
            num_ts: the number of values of t to test for each column
        """
        self.X = X
        self.Y = Y
        self.weights = weights      
        self.n_0 = n_0
        self.frac = frac
        self.num_ts = num_ts
        
        # Initialize the tree with a single node
        # representing all indices of the dataset.
        # The children of node i in self.tree
        # lie at indices 2*i+1 and 2*i+2.
        self.root = Node(X, Y, list(range(len(X))))
        self.tree = [self.root]
        
        self.n_levels = 1 
    
    def split_leaves(self):
        """
            Split each leaf node.
            Returns:
                add_new_nodes: a Boolean saying whether any new
                nodes were created by splitting or not
        """
        
        # The leaves of the tree come at the end
        leaves = self.tree[-2**(self.n_levels-1):]
        
        # An array to hold the new children of the leaves
        children = []
        
        for node in leaves:
            
            # Add the children created by the split for each leaf node.
            # If the node is None then its children will be None.
            if node is not None:
                node.split(self.weights, self.n_0, self.frac, self.num_ts)
                children += [node.lchild, node.rchild]
            else:
                children += [None, None]
        
        # A boolean recording whether any new children have been created
        add_new_nodes = any(children)
        
        # If there are new children, add them to the tree.
        if add_new_nodes:
            self.tree += children
            self.n_levels += 1
        
        return add_new_nodes
        
    def train(self):
        """
            Split all leaf nodes until none can be split further
            according to our end condition.
        """
        continue_splitting = True
        
        while continue_splitting:
            continue_splitting = self.split_leaves()
    
    def predict_flt_one(self, x):
        """
            Predict a float for the array x of shape
            X.shape[1]. This float will be the average
            of the classes for the node in which x lies
            in the decision tree.
        """
        node = self.root
        
        
        # Traverse the decision tree until coming to the
        # node which contains x.
        while node.col:
            if x[node.col] > node.t:
                node = node.rchild
            else:
                node = node.lchild
        
        return node.avg
    
    def predict_one(self, x):
        """
            Predict a class for the array x of
            shape X.shape[1].
        """
        return np.sign(self.predict_flt_one(x))
    
    def predict_flt(self, x):
        """
            Predict floats for the array x with
            x.shape[1] = X.shape[1].
        """
        return np.array(list(map(self.predict_flt_one, x)))
    
    def predict(self, x):
        """
            Predict classes for the array x with
            x.shape[1] = X.shape[1].
        """
        return np.array(list(map(self.predict_one, x)))    

In [46]:
def adaboost_step(X, Y, weights, n_0=500, frac=4, num_ts=20):
    """
        Perform a single step of adaboost. Train a decision
        tree with the given weights. Return the prediction function
        for this decision tree, new weights, and an alpha value.
        X, Y: the dataset
        weights: the weights used for the decision tree
        n_0: the minimum number of data points in a node to split
        frac: the number such that each node has at least n_0 / frac data points
        num_ts: the number of values of t to test for each column
        returns:
            clf: the classifier for the trained tree
            alpha: a weighting for the tree
            new_weights: the new weights to use in the next
                adaboost step
    """
    tree = DecisionTree(X, Y, weights, n_0, frac, num_ts)
    tree.train()
    clf = tree.predict
    Y_pred = clf(X)
    err = np.dot(weights, Y != Y_pred) / weights.sum()
    alpha = np.log((1 - err) / err)
    
    indicator = Y != Y_pred
    new_weights = weights * np.exp(alpha * indicator)
    
    return clf, alpha, new_weights

In [47]:
def adaboost(X, Y, n_steps=20, n_0=500, frac=4, num_ts=20):
    """
        Perform adaboost for a specified number of steps.
        X, Y: the dataset
        n_0: the minimum number of data points in a node to split
        frac: the number such that each node has at least n_0 / frac data points
        num_ts: the number of values of t to test for each column
        returns:
            clfs_coeffs: an array of (clf, alpha) pairs
                where clf is the prediction function for a single
                decision tree and alpha is the weight to use
                for that tree in the adaboost model
    """
    
    # The initial weights are uniform
    init_weights = np.ones(len(Y)) / len(Y)
    weights = init_weights
    clfs_coeffs = []
    
    for j in range(n_steps):
        # Perform a single step of adaboost and print
        # the relevant data from training
        print(f'Step {j+1} / {n_steps}')
        clf, alpha, new_weights = adaboost_step(X, Y, weights, n_0=n_0, frac=frac, num_ts=num_ts)
        print(f'alpha: {alpha}')
        print(f'weights: {new_weights}')
        clfs_coeffs.append((clf, alpha))
        weights = new_weights
    
    return clfs_coeffs

In [48]:
def sign_of_combo_clf(clfs_coeffs):
    """
        Given an array of (clf, alpha) pairs, return a function
        which gives the sign of the sum of weighted terms alpha * clf(x)
        for any input x.
    """
    return lambda x: np.sign(np.sum([alpha * clf(x) for (clf, alpha) in clfs_coeffs], axis=0))

Now we'll perform cross-validation to compare Adaboost and a base tree.

In [49]:
def create_random_splits(X, Y, num_ch):
    """
        Split the dataset (X, Y) into num_ch random chunks
        of equal size (except for possibly the last chunk).
        (X, Y): the dataset
        num_ch: the number of chunks
    """
    
    # Randomly permute (X, Y)
    n = len(X)
    indices = np.random.permutation(n)
    X_perm = X[indices]
    Y_perm = Y[indices]
    
    k = n // num_ch
    
    # Arrays to hold the chunks
    Xs = []
    Ys = []
    
    # Append one chunk at a time
    for i in range(num_ch):
        if i < num_ch-1:
            Xs.append(X_perm[i*k:i*k + k, :])
            Ys.append(Y_perm[i*k:i*k + k])
        else:
            Xs.append(X_perm[i*k:, :])
            Ys.append(Y_perm[i*k:])
    
    return Xs, Ys

In [50]:
# Perform the cross-validation

tree_errors = []
ada_errors = []
num_ch = 5

Xs, Ys = create_random_splits(X, Y, num_ch)

for i in range(num_ch):
    X_left_over = np.concatenate(Xs[:i] + Xs[i+1:], axis=0)
    Y_left_over = np.concatenate(Ys[:i] + Ys[i+1:], axis=0)
    X_ch = Xs[i]
    Y_ch = Ys[i]

    tree = DecisionTree(X_left_over, Y_left_over, n_0=400)
    tree.train()
    tree_clf = tree.predict
    
    clfs_coeffs = adaboost(X_left_over, Y_left_over, n_0=400)
    ada_clf = sign_of_combo_clf(clfs_coeffs)
    
    Y_pred_tree = np.array(tree_clf(X_ch))
    Y_pred_ada = np.array(ada_clf(X_ch))
    
    tree_error = (Y_pred_tree != Y_ch).mean()
    ada_error = (Y_pred_ada != Y_ch).mean()
    
    tree_errors.append(tree_error)
    ada_errors.append(ada_error)

/var/folders/h9/7qb3ygvs0mzb0cdjbh8z8svr0000gn/T/ipykernel_57155/1681302256.py:15: RuntimeWarning: invalid value encountered in scalar divide
  p0 = np.dot(Y[indices] == -1, weights[indices]) / len(Y[indices])
/var/folders/h9/7qb3ygvs0mzb0cdjbh8z8svr0000gn/T/ipykernel_57155/1681302256.py:16: RuntimeWarning: invalid value encountered in scalar divide
  p1 = np.dot(Y[indices] == 1, weights[indices]) / len(Y[indices])


Step 1 / 20
alpha: 2.3970065155187164
weights: [0.00027167 0.00027167 0.00027167 ... 0.00027167 0.00027167 0.00027167]
Step 2 / 20
alpha: 0.7953915819247714
weights: [0.00027167 0.00027167 0.00027167 ... 0.00027167 0.00027167 0.00060182]
Step 3 / 20
alpha: 0.43284005004398907
weights: [0.00027167 0.00027167 0.00027167 ... 0.00027167 0.00027167 0.00092779]
Step 4 / 20
alpha: 0.036245684567853445
weights: [0.00027167 0.00027167 0.00027167 ... 0.00027167 0.00027167 0.00092779]
Step 5 / 20
alpha: 0.0
weights: [0.00027167 0.00027167 0.00027167 ... 0.00027167 0.00027167 0.00092779]
Step 6 / 20
alpha: 0.0
weights: [0.00027167 0.00027167 0.00027167 ... 0.00027167 0.00027167 0.00092779]
Step 7 / 20
alpha: 0.0
weights: [0.00027167 0.00027167 0.00027167 ... 0.00027167 0.00027167 0.00092779]
Step 8 / 20
alpha: 0.0
weights: [0.00027167 0.00027167 0.00027167 ... 0.00027167 0.00027167 0.00092779]
Step 9 / 20
alpha: 0.0
weights: [0.00027167 0.00027167 0.00027167 ... 0.00027167 0.00027167 0.00092779]
S

alpha: 0.0
weights: [0.00027167 0.00043786 0.00027167 ... 0.00027167 0.00027167 0.00062951]
Step 16 / 20
alpha: 0.0
weights: [0.00027167 0.00043786 0.00027167 ... 0.00027167 0.00027167 0.00062951]
Step 17 / 20
alpha: 0.0
weights: [0.00027167 0.00043786 0.00027167 ... 0.00027167 0.00027167 0.00062951]
Step 18 / 20
alpha: 0.0
weights: [0.00027167 0.00043786 0.00027167 ... 0.00027167 0.00027167 0.00062951]
Step 19 / 20
alpha: 0.0
weights: [0.00027167 0.00043786 0.00027167 ... 0.00027167 0.00027167 0.00062951]
Step 20 / 20
alpha: 0.0
weights: [0.00027167 0.00043786 0.00027167 ... 0.00027167 0.00027167 0.00062951]
Step 1 / 20
alpha: 2.24335821908518
weights: [0.00027174 0.00027174 0.00027174 ... 0.00027174 0.00256112 0.00027174]
Step 2 / 20
alpha: 0.9158514966608133
weights: [0.00027174 0.00067905 0.00027174 ... 0.00027174 0.00256112 0.00027174]
Step 3 / 20
alpha: 0.5205437020606252
weights: [0.00027174 0.00067905 0.00027174 ... 0.00045732 0.00256112 0.00045732]
Step 4 / 20
alpha: 0.1056853

In [51]:
print(f'Cross-validation errors for decision tree: {tree_errors}')

Cross-validation errors for decision tree: [0.1, 0.09565217391304348, 0.10760869565217392, 0.10652173913043478, 0.09120521172638436]


In [52]:
print(f'Cross-validation errors for AdaBoost: {ada_errors}')

Cross-validation errors for AdaBoost: [0.1, 0.09565217391304348, 0.10760869565217392, 0.10652173913043478, 0.09120521172638436]


In [53]:
print(f'Mean cross-validation error for decision tree: {np.mean(tree_errors):.5f}')

Mean cross-validation error for decision tree: 0.10020


In [54]:
print(f'Mean cross-validation error for AdaBoost tree: {np.mean(ada_errors):.5f}')

Mean cross-validation error for AdaBoost tree: 0.10020


In this case Adaboost didn't actually improve performance. I'll come back to this later. In particular, the algorithm for choosing a split can be sped up. Speeding up choosing a split and training trees with a smaller number of samples per leaf node might result in better performance.